In [2]:
embeding_matrix = load_pickle_file("./embeding_matrix")
word2index = load_pickle_file("./word2index_titles")
index_sequences_titles = load_pickle_file("./index_sequences_titles.pkl")
index_sequences_text = load_pickle_file("../index_sequences.pkl")
labels = load_pickle_file("../labels")
SEQUENCE_LENGTH = 618
SEQUENCE_LENGTH_TITLE = 10
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3001066

In [3]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences_text]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)

same_lenght_titles = [seq[:SEQUENCE_LENGTH_TITLE].reshape(1,SEQUENCE_LENGTH_TITLE) if len(seq) >= SEQUENCE_LENGTH_TITLE 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH_TITLE - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH_TITLE) 
                   for seq in index_sequences_titles]
same_lenght_titles = np.concatenate(same_lenght_titles,axis=0)
same_lenght_titles = np.array([[int(x) for x in l] for l in same_lenght_titles])


In [4]:
import random
test_size = int(len(labels) * 0.2)
true_labels = np.where(np.array(labels)==0)[0]
start_index = true_labels[0]
random_test_true_indexes = [int(x * len(true_labels)+start_index) for x in np.random.random(test_size//2)]
random_test_false_indexes = [int(x * (len(labels) - len(true_labels))) for x in np.random.random(test_size//2)]
x_test_indexes = random_test_true_indexes + random_test_false_indexes
random.shuffle(x_test_indexes)

x_train_indexes = [x for x in range(0,len(labels)) if x not in x_test_indexes]

random.shuffle(x_train_indexes)


labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

y_test = np.array(labels)[x_test_indexes]
y_train = np.array(labels)[x_train_indexes]

x_train_text = same_lenght_seq[x_train_indexes]
x_train_title = same_lenght_titles[x_train_indexes]

x_test_text = same_lenght_seq[x_test_indexes]
x_test_title = same_lenght_titles[x_test_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  from ipykernel import kernelapp as app


Dataset split

In [5]:
from sklearn.model_selection import train_test_split

x_train_indexes, x_test_indexes, y_train, y_test = train_test_split(list(range(len(same_lenght_seq))), 
                                                                                    labels, test_size=0.2,
                                                                                    random_state=42,
                                                                                    shuffle = True)
x_train_text = same_lenght_seq[x_train_indexes]
x_train_title = same_lenght_titles[x_train_indexes]

x_test_text = same_lenght_seq[x_test_indexes]
x_test_title = same_lenght_titles[x_test_indexes]

In [6]:
#create validation dataset
data_size = len(x_train_text)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train_text = x_train_text[idx_train]
data_train_title = x_train_title[idx_train]

labels_train = y_train[idx_train]

data_val_text = x_train_text[idx_val]
data_val_title = x_train_title[idx_val]

labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>LSTM neural networks</h1>

In [7]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Conv1D, MaxPool1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [7]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [36]:
data_train_text.shape

(11397, 618)

In [8]:
#Text model
LSTM_HIDDEN_DIM_SIZE = 64
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
input_1 = Input(shape=(618,))
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)(input_1)

x1 = LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences = True)(embedding_layer)
x1 = LSTM(LSTM_HIDDEN_DIM_SIZE//2, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)(x1)
x1 = Dense(LSTM_HIDDEN_DIM_SIZE//4, activation = 'tanh')(x1)

#Titles model
LSTM_HIDDEN_DIM_SIZE = 4
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15

input_2 = Input(shape=(10,))
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH_TITLE,
        trainable=False)(input_2)


x2 = LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences = True)(embedding_layer)
x2 = Dropout(0.3)(x2)
x2 = LSTM(LSTM_HIDDEN_DIM_SIZE//2, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)(x2)

concat = tk.layers.Concatenate()([x1,x2])
output = Dense(2,activation='sigmoid')(concat)
full_model = tf.keras.Model(inputs=[input_1, input_2], outputs=[output])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
full_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 618)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 618, 300)     900320100   input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 10, 300)      900320100   input_2[0][0]                    
______________________________________________________________________________________________

In [23]:
full_model.reset_states()

In [10]:
opt = tk.optimizers.Adam(learning_rate=0.02)

In [11]:
full_model.compile(loss='binary_crossentropy',
        optimizer=opt,
        metrics=['acc'])
history2 = full_model.fit([data_train_text,data_train_title],labels_train,epochs = 20,batch_size = 256, 
                     validation_data = [[data_val_text,data_val_title],labels_val],shuffle=True)

Train on 11397 samples, validate on 1267 samples
Epoch 1/20
11397/11397 [==============================] - 233s 20ms/sample - loss: 0.5700 - acc: 0.7108 - val_loss: 0.4118 - val_acc: 0.8327
Epoch 2/20
11397/11397 [==============================] - 234s 21ms/sample - loss: 0.3443 - acc: 0.8780 - val_loss: 0.3053 - val_acc: 0.8879
Epoch 3/20
11397/11397 [==============================] - 226s 20ms/sample - loss: 0.2831 - acc: 0.8929 - val_loss: 0.2285 - val_acc: 0.9033
Epoch 4/20
11397/11397 [==============================] - 222s 19ms/sample - loss: 0.2759 - acc: 0.9004 - val_loss: 0.2302 - val_acc: 0.9029
Epoch 5/20
11397/11397 [==============================] - 224s 20ms/sample - loss: 0.2631 - acc: 0.9040 - val_loss: 0.2619 - val_acc: 0.9088
Epoch 6/20
11397/11397 [==============================] - 222s 20ms/sample - loss: 0.2634 - acc: 0.9048 - val_loss: 0.2592 - val_acc: 0.9084
Epoch 7/20
11397/11397 [==============================] - 223s 20ms/sample - loss: 0.2585 - acc: 0.9064 -

In [14]:
history2 = full_model.fit([data_train_text,data_train_title],labels_train,epochs = 5,batch_size = 256, 
                     validation_data = [[data_val_text,data_val_title],labels_val],shuffle=True)

Train on 11397 samples, validate on 1267 samples
Epoch 1/5
11397/11397 [==============================] - 270s 24ms/sample - loss: 0.2001 - acc: 0.9264 - val_loss: 0.2245 - val_acc: 0.9171
Epoch 2/5
11397/11397 [==============================] - 269s 24ms/sample - loss: 0.1967 - acc: 0.9238 - val_loss: 0.2294 - val_acc: 0.9140
Epoch 3/5
11397/11397 [==============================] - 244s 21ms/sample - loss: 0.1949 - acc: 0.9267 - val_loss: 0.2293 - val_acc: 0.9132
Epoch 4/5
11397/11397 [==============================] - 231s 20ms/sample - loss: 0.1898 - acc: 0.9294 - val_loss: 0.2296 - val_acc: 0.9128
Epoch 5/5
11397/11397 [==============================] - 235s 21ms/sample - loss: 0.1881 - acc: 0.9298 - val_loss: 0.2269 - val_acc: 0.9203


In [21]:
conv_model.save("./Checkpoints/conv_model2.h5")

In [22]:
conv_model.save_weights("./Checkpoints/conv_weights2")

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = full_model.predict([x_test_text,x_test_title])
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

F1-score : 0.9384816753926701
Accuracy-score : 0.9109567413956425
Precision : 0.9056842105263158
Recall  : 0.9737437754640109


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = full_model.predict([x_test_text,x_test_title])
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

In [13]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.8699616580869116

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

In [29]:
from sklearn.metrics import accuracy_score

pred = conv_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [30]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)

0.9362930077691454

In [31]:
accuracy_score([np.argmax(x) for x in y_test],pred)

0.9093779602147143

In [32]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.8797661587489404

In [34]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")

Precision : 0.8851894374282434
Recall  : 0.8048016701461378


In [1]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
